**SA463A &#x25aa; Data Wrangling and Visualization &#x25aa; Fall 2021 &#x25aa; Uhan**

# Lesson 16. Split-Apply-Combine in Pandas

## In this lesson...

- One key task in data wrangling is to *split* our dataset into groups, *perform* a computation on each group, and *combine* the results


- This is often referred to as the **split-apply-combine** paradigm:
    * **Split** the data into groups based on some criteria
    * **Apply** a function to each group independently
    * **Combine** the results


- The apply step might involve one of the following:
    - aggregation
    - transformation
    - filtration


- Let's see how this works

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Setup

* Let's start by importing Pandas:

In [1]:
import pandas as pd

* We'll use the nycflights13 dataset that we used in previous lessons, containing data on all flights outbound from NYC airports in 2013


* This data is located in `data/nycflights13_flights.csv.zip`, in the same folder as this notebook:

In [2]:
df = pd.read_csv('data/nycflights13_flights.csv.zip')

* Just to remind ourselves what this dataset looks like:

In [3]:
df.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T10:00:00Z
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01T10:00:00Z
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z


<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Aggregation

- **Aggregation** involves computing a summary statistic for each group of observations


- Visually:

<img src="img/aggregation.jpg" width="800"/>


- To make this concrete, let's work on an example with the nycflights13 dataset


- Suppose we want to explore the relationship between the distance traveled and arrival delay for each destination


- We can do the following:
    1. Group flights by destination
    2. Compute average distance, average arrival delay, and number of flights for each destination
    3. Filter to remove noise: only keep values for destinations with 20 or more flights, remove Honolulu

In [4]:
# Solution
delay_df = (
    df
    .groupby(['dest'])
    .agg(
        avg_distance=('distance', 'mean'),
        avg_arr_delay=('arr_delay', 'mean'),
        n_flights=('flight', 'count')  # Every flight has a flight number (no missing values)
    )
    .reset_index()
    .query('(n_flights > 20) and (dest != "HNL")')
)

delay_df.head(10)

,dest,avg_distance,avg_arr_delay,n_flights
0,ABQ,1826.000000,4.381890,254
1,ACK,199.000000,4.852273,265
2,ALB,143.000000,14.397129,439
4,ATL,757.108220,11.300113,17215
5,AUS,1514.252973,6.019909,2439
6,AVL,583.581818,8.003831,275
7,BDL,116.000000,7.048544,443
8,BGR,378.000000,8.027933,375
9,BHM,865.996633,16.877323,297
10,BNA,758.213485,11.812459,6333


- *Quick aside.* Note that `.head(n)` returns the top `n` rows of a DataFrame
    - By default, `.head()` returns the top 5 rows

- Let's walk through this code, step-by-step


- First, `.groupby(['dest'])` *splits* the DataFrame `df` into groups of rows, according to the row's value of `dest`


- Next, `.agg(...)` *applies* various functions to each group independently
    - The keyword argument
    
        ```python
        avg_distance=('distance', 'mean')

        ```
        
        <br>outputs a variable called `avg_distance`, which is equal to the Pandas built-in function `mean` applied to the column `distance` *in each group*
        
    - The other keyword arguments above work in a similar fashion
    
    
- `.agg(...)` then takes the output from each group and *combines* them into a single output DataFrame, where the index is equal to the variables specified in `.groupby(...)`


- `.reset_index()` converts the existing index into ordinary columns, and resets the index of the DataFrame to the default one (consecutive integers)
    - This is often desired when performing additional wrangling or analysis steps


- `.query('(n_flights > 20) and (dest != "HNL")')` filters the rows of the output DataFrame

- **Basic template for aggregation:**

    ```python
    summarized_df = (
        df
        .groupby(list_of_variables)
        .agg(
            new_variable1=('variable1', 'aggregation function'),
            new_variable2=('variable2', 'aggregation function'),
            ...
        )
        .reset_index()
        ...
    )
    ```


- Here are some common aggregation functions
    - Note that many of them are *reduction methods* from Lesson 15
    
    
| Method | Description |
| :- | :- |
| `count` | Number of non-NA values |
| `first` | First value in group |
| `last` | Last value in group |
| `nunique` | Number of unique values |
| `min`, `max` | Minimum and maximum values |
| `sum` | Sum of values |
| `mean` | Mean of values |
| `median` | Median of values |
| `mad` | Mean absolute deviation from mean value |
| `prod` | Product of all values |
| `var` | Sample variance of values |
| `std` | Sample standard deivation of values |


- Instead of the `'aggregation function'` string, you can pass a function itself

    - For example, instead of `('variable1', 'sum')`, you could use<br><br>
    
        ```python
        ('variable1', lambda s: s.sum())
        ```
        
        <br>where `s` refers to the `variable1` column/Series of an *individual group* 


- *Note.* This method of using `.agg()` is called *named aggregation*
    - There are a few other ways of using `.agg()`
    - See the [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#aggregation) for details

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Grouping by multiple variables

- We can split a DataFrame into groups based on the values of *multiple* variables


- For example, we can compute the number of flights on each day like this:

In [5]:
# Solution
per_day = (
    df
    .groupby(['year', 'month', 'day'])
    .agg(
        flights=('flight', 'count')
    )
    .reset_index()
)

per_day.head(10)

,year,month,day,flights
0,2013,1,1,842
1,2013,1,2,943
2,2013,1,3,914
3,2013,1,4,915
4,2013,1,5,720
5,2013,1,6,832
6,2013,1,7,933
7,2013,1,8,899
8,2013,1,9,902
9,2013,1,10,932


<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Transformation

- **Transformation** involves performing a computation on each group, and returning objects that are the same size as each group, with the same index


- Visually:

<img src="img/transformation.jpg" width="800"/>


- Transformation usually involves a "same size" method like we described in Lesson 15


- For example, recall that `.rank()` is a "same size" method that computes numerical ranks (for example, 1 = highest, 2 = second highest, etc.)


- For each day of the year, we can compute the rank of each flight according to its arrival delay, from highest (rank = 1) to lowest, like this:

In [6]:
# Solution
(
    df
    .groupby(['year', 'month', 'day'])
    ['arr_delay']
    .transform(lambda s: s.rank(ascending=False))
)

0         296.5
1         202.0
2         123.5
3         746.5
4         792.0
          ...  
336771      NaN
336772      NaN
336773      NaN
336774      NaN
336775      NaN
Name: arr_delay, Length: 336776, dtype: float64

<br>❓ **Exercise 1.** The first flight (with index 0) has the 296.5th highest arrival delay among all flights on the same day. What do you think a fractional rank means? 
*Hint.* [Here's the documentation for `.rank()`.](https://pandas.pydata.org/docs/reference/api/pandas.Series.rank.html)

*Write your notes here. Double-click to edit.*

*Solution.* By default, for a group of records that have the same value (i.e. ties), the `.rank()` method reports the average rank of the group

- Let's walk through this code step-by-step


- `.groupby(['year', 'month', 'day'])` splits the DataFrame `df` into groups of rows, according to the combination of values of `year`, `month`, and `day`


- `['arr_delay']` then selects the `arr_delay` column/Series


- `.transform(...)` then takes the `arr_delay` Series *from each group* and applies the lambda function to it


- In `lambda s: s.rank(ascending=False)`:
    - `s` represents the `arr_delay` column/Series from each group
    - The lambda function outputs the numerical rank for each value in the Series, with the highest value having rank 1


- We can use the `.assign()` method to add these numerical ranks to our original DataFrame, like this:

In [7]:
# Solution
new_df = df.assign(
    daily_arr_delay_rank=lambda x:
        x.groupby(['year', 'month', 'day'])
        ['arr_delay']
        .transform(lambda s: s.rank(ascending=False))
)

new_df.head(10)

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour,daily_arr_delay_rank
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z,296.5
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z,202.0
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T10:00:00Z,123.5
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01T10:00:00Z,746.5
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z,792.0
5,2013,1,1,554.0,558,-4.0,740.0,728,12.0,UA,1696,N39463,EWR,ORD,150.0,719,5,58,2013-01-01T10:00:00Z,279.0
6,2013,1,1,555.0,600,-5.0,913.0,854,19.0,B6,507,N516JB,EWR,FLL,158.0,1065,6,0,2013-01-01T11:00:00Z,211.0
7,2013,1,1,557.0,600,-3.0,709.0,723,-14.0,EV,5708,N829AS,LGA,IAD,53.0,229,6,0,2013-01-01T11:00:00Z,703.5
8,2013,1,1,557.0,600,-3.0,838.0,846,-8.0,B6,79,N593JB,JFK,MCO,140.0,944,6,0,2013-01-01T11:00:00Z,616.5
9,2013,1,1,558.0,600,-2.0,753.0,745,8.0,AA,301,N3ALAA,LGA,ORD,138.0,733,6,0,2013-01-01T11:00:00Z,339.5


❓ **Exercise 2.** Let's pick a random day of the year and check our work.
Filter the observations for all flights on June 12, 2013, and sort them in descending order of arrival delay. Display the top 10 flights. Do their values of `daily_arr_delay_rank` make sense?

In [8]:
# Solution
(
    new_df
    .query('(year == 2013) and (month == 6) and (day == 12)')
    .sort_values('arr_delay', ascending=False)
    .head(10)
)

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour,daily_arr_delay_rank
233351,2013,6,12,2332.0,2030,182.0,137.0,2155,222.0,WN,1271,N233LV,EWR,MDW,131.0,711,20,30,2013-06-13T00:00:00Z,1.0
233336,2013,6,12,2153.0,1905,168.0,25.0,2108,197.0,9E,3525,N921XJ,JFK,ORD,176.0,740,19,5,2013-06-12T23:00:00Z,2.0
232763,2013,6,12,1151.0,836,195.0,1300.0,947,193.0,B6,1172,N292JB,EWR,BOS,46.0,200,8,36,2013-06-12T12:00:00Z,3.0
233331,2013,6,12,2143.0,1820,203.0,2347.0,2052,175.0,9E,3542,N914XJ,JFK,MSP,153.0,1029,18,20,2013-06-12T22:00:00Z,4.0
232784,2013,6,12,1202.0,920,162.0,1427.0,1210,137.0,AA,1223,N583AA,EWR,DFW,180.0,1372,9,20,2013-06-12T13:00:00Z,5.0
233341,2013,6,12,2212.0,1935,157.0,2353.0,2137,136.0,EV,5550,N759EV,EWR,DTW,76.0,488,19,35,2013-06-12T23:00:00Z,6.5
233142,2013,6,12,1807.0,1620,107.0,2201.0,1945,136.0,AA,1467,N3CBAA,LGA,MIA,173.0,1096,16,20,2013-06-12T20:00:00Z,6.5
233332,2013,6,12,2145.0,2030,75.0,32.0,2217,135.0,B6,917,N334JB,JFK,ORD,182.0,740,20,30,2013-06-13T00:00:00Z,8.0
233178,2013,6,12,1835.0,1640,115.0,2202.0,1956,126.0,B6,185,N565JB,JFK,SAN,311.0,2446,16,40,2013-06-12T20:00:00Z,9.0
233101,2013,6,12,1727.0,1600,87.0,2129.0,1926,123.0,DL,1373,N915DE,JFK,MIA,166.0,1089,16,0,2013-06-12T20:00:00Z,10.0


* **Basic template for transformation:**

    ```python
    df_with_transformed_column = df.assign(
        new_variable=lambda x: 
            x.groupby(list_of_variables)
            ['variable']
            .transform(lambda s: s.same_size_method(...))
    )
    ```
    
    <br>where `x` refers to `df`, and `s` refers to `x['variable']`

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Filtration

- **Filtration** allows us to keep or discard some groups based on a group-wise computation that evaluates to `True` or `False`


- Visually:

<img src="img/filtration.jpg" width="800"/>


- For example, we can keep all flights to destination airports with 10,000 or more total flights, like this:

In [9]:
# Solution
busy_dest_df = (
    df
    .groupby(['dest'])
    .filter(lambda x: x['flight'].count() > 10000)
)

busy_dest_df

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T10:00:00Z
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z
5,2013,1,1,554.0,558,-4.0,740.0,728,12.0,UA,1696,N39463,EWR,ORD,150.0,719,5,58,2013-01-01T10:00:00Z
6,2013,1,1,555.0,600,-5.0,913.0,854,19.0,B6,507,N516JB,EWR,FLL,158.0,1065,6,0,2013-01-01T11:00:00Z
8,2013,1,1,557.0,600,-3.0,838.0,846,-8.0,B6,79,N593JB,JFK,MCO,140.0,944,6,0,2013-01-01T11:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336754,2013,9,30,2147.0,2137,10.0,30.0,27,3.0,B6,1371,N627JB,LGA,FLL,139.0,1076,21,37,2013-10-01T01:00:00Z
336755,2013,9,30,2149.0,2156,-7.0,2245.0,2308,-23.0,UA,523,N813UA,EWR,BOS,37.0,200,21,56,2013-10-01T01:00:00Z
336762,2013,9,30,2233.0,2113,80.0,112.0,30,42.0,UA,471,N578UA,EWR,SFO,318.0,2565,21,13,2013-10-01T01:00:00Z
336763,2013,9,30,2235.0,2001,154.0,59.0,2249,130.0,B6,1083,N804JB,JFK,MCO,123.0,944,20,1,2013-10-01T00:00:00Z


- How does this code work?


- First, `.groupby(['dest'])` splits the DataFrame `df` into groups of rows, according to the row's value of `dest`


- Then, `.filter(...)` takes a function that, when applied to the group as a whole, returns True or False


- In `lambda x: x['flight'].count() > 10000`:
    - `x` represents a single group
    - `x['flight'].count()` counts the non-NA values in the `flight` column of each group `x`

❓ **Exercise 3.** Let's check our work. Using the techniques from the *Aggregation* section above, count the number of flights to each destination in `busy_dest_df`. Does what you see make sense?

In [10]:
# Solution
(
    busy_dest_df
    .groupby(['dest'])
    .agg(
        n_flights=('flight', 'count')
    )
)

,n_flights
dest,
ATL,17215
BOS,15508
CLT,14064
FLL,12055
LAX,16174
MCO,14082
MIA,11728
ORD,17283
SFO,13331


* **Basic template for filtration:**

    ```python
    filtered_df = (
        df
        .groupby(list_of_variables)
        .filter(lambda x: expression that evaluates to True or False)
    )
    ```
    
    <br>where `x` refers to `df`

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Related tips and tricks

### Aggregation on the entire DataFrame 

- As we've seen above, the split-apply-combine paradigm gives us a nice, standard template for computing summary statistics for groups of observations in a DataFrame (i.e., aggregation)


- We can also use the same template for computing summary statistics for *all* the observations in a DataFrame 


- Suppose we want the average, median, minimum, and maximum arrival delay across all flights in 2013


- We can do so like this:

In [11]:
# Solution
(
    df
    .groupby(lambda r: True)
    .agg(
        average_arr_delay=('arr_delay', 'mean'),
        median_arr_delay=('arr_delay', 'median'),
        min_arr_delay=('arr_delay', 'min'),
        max_arr_delay=('arr_delay', 'max')
    )
    .reset_index(drop=True)
)

,average_arr_delay,median_arr_delay,min_arr_delay,max_arr_delay
0,6.895377,-5.0,-86.0,1272.0


- A few variations from the standard aggregation template...


- We give `.groupby()` the argument `lambda r: True` to put all the rows into one group 


- Why does this work? 
    - `.groupby()` can take a function `lambda r: ...` as an argument, where `r` represents a row index
    - When given such an argument, `.groupby()` forms groups according to the output values of this lambda function
    - In the code above, *every* row index returns the value `True`, so every row gets put into the same group 


- We also use `drop=True` in `.reset_index()`, since we don't need the name of the group in this case

### Counting unique values and `.value_counts()`

- We can use the above techniques to count the unique values in a particular column


- For example, we can count the number of flights by each airline like this:

In [12]:
# Solution
(
    df
    .groupby(['carrier'])
    .agg(
        n_flights=('flight', 'count')
    )
    .reset_index()
    .sort_values('n_flights', ascending=False)
)

,carrier,n_flights
11,UA,58665
3,B6,54635
5,EV,54173
4,DL,48110
1,AA,32729
9,MQ,26397
12,US,20536
0,9E,18460
14,WN,12275
13,VX,5162


- You can also get a similar table of counts using the `.value_counts()` method, like this:

In [13]:
# Solution
df['carrier'].value_counts()

UA    58665
B6    54635
EV    54173
DL    48110
AA    32729
MQ    26397
US    20536
9E    18460
WN    12275
VX     5162
FL     3260
AS      714
F9      685
YV      601
HA      342
OO       32
Name: carrier, dtype: int64

- Using `.value_counts()` can be quick and convenient when all you need is to count the number of unique values in a column/Series


- If you need additional summary statistics, the standard aggregation template using `.groupby(...).agg(...)` may be easier to use

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Problems

For the problems below, use the nycflights13 dataset that we used in this lesson.

### Problem 1

What time of day should you fly if you want to avoid delays as much as possible? Compute the average departure and arrival delays for each hour of the day.

In [14]:
# Solution
(
    df
    .groupby(['hour'])
    .agg(
        avg_dep_delay=('dep_delay', 'mean'),
        avg_arr_delay=('arr_delay', 'mean')
    )
    .reset_index()
)

,hour,avg_dep_delay,avg_arr_delay
0,1,NaN,NaN
1,5,0.687757,-4.796907
2,6,1.642796,-3.384485
3,7,1.914078,-5.304472
4,8,4.127948,-1.113227
5,9,4.583738,-1.451407
6,10,6.498295,0.953940
7,11,7.191650,1.481930
8,12,8.614849,3.489010
9,13,11.437650,6.544740


### Problem 2

Assume that flights with missing arrival delays correspond to cancelled flights. Compute the number of cancelled flights for each month.

*Hint.* Use `.query()` to narrow down the dataset to rows corresponding to cancelled flights.

In [15]:
# Solution
(
    df
    # engine='python' may not be necessary, depending on your setup
    .query('arr_delay.isna()', engine='python')
    .groupby(['year', 'month'])
    .agg(
        n_cancelled=('flight', 'count')
    )
    .reset_index()
)

,year,month,n_cancelled
0,2013,1,606
1,2013,2,1340
2,2013,3,932
3,2013,4,766
4,2013,5,668
5,2013,6,1168
6,2013,7,1132
7,2013,8,571
8,2013,9,564
9,2013,10,271


### Problem 3

Which major carrier has the worst delays? Compute the average arrival delay for each carrier that has more than 10,000 flights in 2013. Sort them from highest to lowest average arrival delay.

In [16]:
# Solution
(
    df
    .groupby(['carrier'])
    .filter(lambda x: x['flight'].count() > 10000)
    .groupby(['carrier'])
    .agg(
        avg_arr_delay=('arr_delay', 'mean')
    )
    .reset_index()
    .sort_values('avg_arr_delay', ascending=False)
)

,carrier,avg_arr_delay
4,EV,15.796431
5,MQ,10.774733
8,WN,9.649120
2,B6,9.457973
0,9E,7.379669
6,UA,3.558011
7,US,2.129595
3,DL,1.644341
1,AA,0.364291


### Problem 4

Perhaps it's a better idea to compare the average arrival delay between carriers on the same route.

Focus on routes from a NYC airport (EWR, JFK, LGA) to either Atlanta (ATL) or Chicago O'Hare (ORD). For each route, find the carrier with the highest average arrival delay.

*Hint.* Use `.agg()`, sort the resulting values, and then use `.agg()` again.

In [17]:
# Solution
(
    df
    .query('dest in ["ATL", "ORD"]')
    .groupby(['origin', 'dest', 'carrier'])
    .agg(
        avg_arr_delay=('arr_delay', 'mean')
    )
    .reset_index()
    .sort_values(['avg_arr_delay'], ascending=False)
    .groupby(['origin', 'dest'])
    .agg(
        worst_carrier=('carrier', 'first'),
        worst_delay=('avg_arr_delay', 'first')
    )
    .reset_index()
)

,origin,dest,worst_carrier,worst_delay
0,EWR,ATL,EV,19.546554
1,EWR,ORD,EV,17.500000
2,JFK,ATL,EV,128.000000
3,JFK,ORD,B6,14.869955
4,LGA,ATL,EV,63.000000
5,LGA,ORD,OO,107.000000


### Problem 5

Consider the following code, that compares the `dep_delay` column with the value `0`:

In [18]:
df['dep_delay'] > 0

0          True
1          True
2          True
3         False
4         False
          ...  
336771    False
336772    False
336773    False
336774    False
336775    False
Name: dep_delay, Length: 336776, dtype: bool

Here we have another way of creating a new variable in Pandas! Note that the result is another Series, with the same index as `df['dep_delay']`. The values in the Series are `True` or `False`, depending on whether the value of `dep_delay` is positive.

In Pandas, we can convert these `True` and `False` values to integers, using the `.astype()` method. Note that `True` is converted to `1`, and `False` is converted to `0`:

In [19]:
(df['dep_delay'] > 0).astype(int)

0         1
1         1
2         1
3         0
4         0
         ..
336771    0
336772    0
336773    0
336774    0
336775    0
Name: dep_delay, Length: 336776, dtype: int64

In this way, we can create a new variable that represents whether a flight left on time.

How often does a plane (tail number) leave on time? For each plane, compute the fraction of flights it leaves on time.

In [20]:
# Solution
(
    df
    .assign(
        left_on_time=lambda x: (x['dep_delay'] <= 0).astype(int)
    )
    .groupby(['tailnum'])
    .agg(
        frac_left_on_time=('left_on_time', 'mean')
    )
    .reset_index()
)

,tailnum,frac_left_on_time
0,D942DN,0.250000
1,N0EGMQ,0.668464
2,N10156,0.483660
3,N102UW,0.770833
4,N103US,0.847826
...,...,...
4038,N997DL,0.603175
4039,N998AT,0.423077
4040,N998DL,0.545455
4041,N999DN,0.721311


### Problem 6

For each plane (tail number), compute its cumulative departure delay over each day in 2013 that it departed from a NYC airport.

*Hint.* First, remove all flights with missing departure delays.

In [21]:
# Solution
(
    df
    [['tailnum', 'year', 'month', 'day', 'dep_delay']]
    # engine='python' may not be necessary, depending on your setup
    .query('not dep_delay.isna()', engine='python')
    .sort_values(['tailnum', 'year', 'month', 'day'])
    .assign(
        cum_delay=lambda x: 
            x.groupby(['tailnum'])
            ['dep_delay']
            .transform(
                lambda s: s.cumsum()
            )
    )
)

,tailnum,year,month,day,dep_delay,cum_delay
120316,D942DN,2013,2,11,68.0,68.0
157233,D942DN,2013,3,23,40.0,108.0
157799,D942DN,2013,3,24,24.0,132.0
254418,D942DN,2013,7,5,-6.0,126.0
523,N0EGMQ,2013,1,1,54.0,54.0
...,...,...,...,...,...,...
104430,N9EAMQ,2013,12,23,169.0,2265.0
105134,N9EAMQ,2013,12,24,9.0,2274.0
107510,N9EAMQ,2013,12,27,-8.0,2266.0
109161,N9EAMQ,2013,12,29,35.0,2301.0


<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Notes and sources

- From the [Pandas User Guide](https://pandas.pydata.org/docs/user_guide/index.html):
    - [Group by: split-apply-combine](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html)


- Lesson and problems inspired by Chapter 5 of [R for Data Science](https://r4ds.had.co.nz/)